In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

# 예시 데이터 (아이템의 콘텐츠 정보: 제목과 설명)
items = {
    '아이템': ['기생충', '부산행', '태극기 휘날리며', '도둑들', '설국열차', '범죄도시'],
    '설명': [
        '가난한 가족이 부잣집에 얽히며 벌어지는 이야기',
        '좀비 바이러스가 퍼진 세상에서 살아남기 위한 이야기',
        '한국 전쟁을 배경으로 형제의 이야기를 그린 영화',
        '도둑들이 모여 큰 한탕을 노리는 이야기',
        '미래의 기차 안에서 벌어지는 계급 투쟁 이야기',
        '형사가 조직 범죄와 싸우는 액션 영화'
    ]
}

ratings = {
    '사용자': ['사용자 A', '사용자 A', '사용자 A', '사용자 B', '사용자 B', '사용자 C'],
    '아이템': ['기생충', '부산행', '설국열차', '태극기 휘날리며', '도둑들', '범죄도시'],
    '평점': [5, 4, 3, 5, 3, 4]
}

# 데이터프레임으로 변환
df_items = pd.DataFrame(items)
df_ratings = pd.DataFrame(ratings)

# TF-IDF 벡터라이저를 사용하여 설명을 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_items['설명'])

# 사용자-아이템 평점 매트릭스 생성
user_item_matrix = df_ratings.pivot(index='사용자', columns='아이템', values='평점').fillna(0)

# K-means 클러스터링을 사용하여 사용자 클러스터링
num_clusters = 2
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
user_clusters = kmeans.fit_predict(user_item_matrix)

# 클러스터링 결과를 데이터프레임에 추가
user_item_matrix['클러스터'] = user_clusters

# 특정 클러스터의 다른 사용자들이 높게 평가한 아이템 추천
def cluster_based_recommendations(user_id, num_recommendations):
    user_cluster = user_item_matrix.loc[user_id, '클러스터']
    similar_users = user_item_matrix[user_item_matrix['클러스터'] == user_cluster].drop(user_id)
    
    # 모든 사용자의 아이템 평점 평균 계산
    avg_ratings = similar_users.mean(axis=0).drop('클러스터')
    
    # 이미 평가한 아이템 제외
    user_rated_items = user_item_matrix.loc[user_id].drop('클러스터')
    user_rated_items = user_rated_items[user_rated_items > 0].index
    avg_ratings = avg_ratings.drop(user_rated_items)
    
    # 상위 num_recommendations개의 아이템 추천
    recommendations = avg_ratings.nlargest(num_recommendations).index.tolist()
    
    return recommendations

# 사용자 A에 대한 추천 계산 (예: '사용자 A'에게 유사한 아이템 3개 추천)
recommended_items = cluster_based_recommendations('사용자 A', 3)

print(f"사용자 A에게 추천하는 아이템:")
for item in recommended_items:
    print(item)


사용자 A에게 추천하는 아이템:
도둑들
범죄도시
태극기 휘날리며
